In [19]:
import numpy as np

def fast_matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    R[np.isnan(R)] = 0
    
    r_plus_mask = np.where(R > 0)
    
    eij = R - np.dot(P, Q.T)
    eij[R <= 0] = 0
    r_plus_bin = np.sign(eij)
    
    PK_mask = np.repeat(np.sum(r_plus_bin, axis=1, keepdims=True), K, axis=1)
    QK_mask = np.repeat(np.sum(r_plus_bin, axis=0, keepdims=True), K, axis=0)
    
    for step in range(steps):
        if step % 50 == 0:
            print(step)
            
        eij = R - np.dot(P, Q.T)
        eij[R <= 0] = 0
                
        P += 2 * alpha * np.dot(eij, Q) - alpha * beta * np.multiply(P, PK_mask)
        Q += 2 * alpha * np.dot(eij.T, P) - alpha * beta * np.multiply(Q.T, QK_mask).T

#         eR = np.dot(P, Q.T)
        e_loss = np.sum(np.power(R - np.dot(P, Q.T), 2)[r_plus_mask])
        e_reg = (beta/2) * (np.sum(np.multiply(np.power(P, 2), PK_mask)) + np.sum(np.multiply(np.power(Q.T, 2), QK_mask)))
#         e = e_loss + e_reg

#         if step % 500 == 0:
#             print(e_reg)
#             print("Iter {0}: LOSS: {1:.6f}, REG: {2:.6f}".format(step / 50, e_loss, e_reg))

        if e_loss + e_reg < 0.001:
            break
    
    return P, Q

In [6]:
import csv
import pandas

dfApp = pandas.read_csv('data/app_dataframe_TRAIN.csv')
dfArt = pandas.read_csv('data/article.csv')

# article = list(dfArt)
# article = article[1:]
# category = list(dfApp)
# category = category[3:]
user = dfApp.user_code.tolist()
uid = dfApp.user_id.tolist()    

#delete the column names and userids/usercodes, only keep the freq values
# Uapp = dfApp.values
# Uapp = numpy.delete(Uapp, numpy.s_[0:3], 1)

#delete the column names and userids/usercodes, only keep the dwelltime values
# dfArt.dropna(axis=1, how='all')
# Uart = dfArt.values
# Uart = numpy.delete(Uart, 0, 1)
#delete articles with all nan values
dfArtP = dfArt.dropna(axis=1, how='all')
articleP = list(dfArtP)
articleP = articleP[1:]
Uartp = dfArtP.values
Uartp = numpy.delete(Uartp, 0, 1)
Uartp = Uartp[:20]
#to speed up, only use the top-20 to predict
user = user[:20]
uid = uid[:20]
# Uapp = Uapp[:20]
# Uart = Uart[:20]

# print(Uartp)


NameError: name 'numpy' is not defined

In [20]:
Uartp = np.delete(Uartp, 0, 1)
Uartp = Uartp[:20]
#to speed up, only use the top-20 to predict
user = user[:20]
uid = uid[:20]
# Uapp = Uapp[:20]
# Uart = Uart[:20]

# print(Uartp)


In [21]:
M = len(Uartp)
N = len(Uartp[0])
K = 2

_P = np.random.rand(M,K)
_Q = np.random.rand(N,K)

In [22]:
# P1 = _P.copy()
# P2 = _P.copy()
# P3 = _P.copy()
# Q1 = _Q.copy()
# Q2 = _Q.copy()
# Q3 = _Q.copy()

In [23]:
import time

# start_time = time.time()
nP, nQ = fast_matrix_factorization(Uartp, _P, _Q, K)
nR2 = np.dot(nP, nQ.T)
# end_time = time.time() - start_time
print(nR2)
# print("Fast:", end_time)

# print(np.sum(np.abs(nR1 - nR2)) / (M*N))

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
[[ 4.91450165  4.14922043  2.40188897 ...,  9.54200139  9.09994687
   5.88800749]
 [ 4.32353691  4.13327077  2.06224591 ...,  8.14758383  7.85162029
   5.41465448]
 [ 5.4363159   7.76044732  2.32331556 ...,  8.93368679  9.05477752
   8.05374847]
 ..., 
 [ 0.46352973  0.54274629  0.21061395 ...,  0.82256589  0.81000278
   0.62891018]
 [ 4.24432703  3.22615526  2.11193922 ...,  8.42348587  7.972973
   4.91149905]
 [ 3.50036048  2.44234546  1.76471798 ...,  7.05862053  6.64506916
   3.94451341]]


In [30]:
article = list(dfArt)
article = article[1:]
Uart = dfArt.values
Uart = np.delete(Uart, 0, 1)

In [33]:
nR = nR2.copy()

import math
topR1 = []
for i in range(nR.shape[0]):
    lineR1 = []
    lineR2 = []
    #return the index with ranked 1~5
    subRank = nR[i].argsort()[-5:][::-1]
    subRank2 = Uart[i].argsort()
    print ('user #',user[i],uid[i])
#     print subRank
    for j in range(5):
        tmp = articleP[subRank[j]]
        tmp2 = article[subRank2[j]]
        lineR1.append(tmp)
    for k in range(nR.shape[0]):
        if math.isnan(Uart[i][subRank2[k]]):
            continue
        tmp2 = article[subRank2[k]]
        lineR2.append(tmp2)
        if len(lineR2)==5:
            break
    print (lineR1)
    print (lineR2)
    
# print topR

user # 0 00108FC5F1F3031ADD536548124A4912
['39169', '2518', '19069', '6570', '41177']
['22344']
user # 1 00310455B7B24D5CDEC892351B8D1D51
['39169', '2518', '19069', '6570', '10523']
['40351']
user # 2 00381B68ACD6CC83635CDDD9B40F0BAF
['39169', '2518', '10523', '19069', '6570']
['12897', '16891', '35968', '19071', '40613']
user # 3 003B2215B070CDB50C0819C72E208F0A
['39169', '2518', '10523', '19069', '6570']
['41089', '4367', '8597', '4283', '13976']
user # 4 003B8D414199B9C5EB7CF8C94BCC6787
['39169', '2518', '19069', '10523', '6570']
[]
user # 5 0043B1ADB92D7265FCF0B744A42AAF05
['39169', '2518', '19069', '10523', '6570']
['34968']
user # 6 00450B9C8D0055C699C03D6A0B1F6CE8
['39169', '2518', '19069', '10523', '6570']
[]
user # 7 0049D0D7726B7A1DEFB26EFE6B336B38
['39169', '2518', '19069', '10523', '6570']
[]
user # 8 004A20E76B5AFEBF06EC46D0FD33A404
['39169', '2518', '19069', '6570', '41177']
['46595', '22512', '5905', '34612', '37315']
user # 9 004A5982BCE2DECD0622672F6DFE488E
['39169', '

In [15]:
print (np.nan <= 0)
print (np.nan + 8)

False
nan
